# 动量策略实战：时序对齐、成本与参数敏感性

## 目的
构建并评估经典动量策略（SMA交叉和时间序列动量），掌握严格时序对齐、交易成本处理，并进行参数敏感性分析。

## 金融含义  
动量策略基于"趋势延续"假设，是量化交易的核心策略族。通过对比不同动量信号的表现，理解策略设计、成本控制和参数优化的重要性。


## 1. 导入库与数据获取

### 目的
导入必要的库和自定义工具函数，下载AAPL股票数据作为测试标的。

### 金融含义
选择流动性好的大盘股（AAPL）作为策略测试标的，确保数据质量和交易的可执行性。


In [1]:
# 导入必要库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

# 导入自定义工具函数
from src.miniqlib import ensure_dir, shift_exec, turnover, apply_cost, equity, perf, max_drawdown

# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("库导入完成")


库导入完成


In [ ]:
# 下载AAPL数据
try:
    data = yf.download('AAPL', start='2015-01-01', end=datetime.now().strftime('%Y-%m-%d'), auto_adjust=True)
    px = data['Close'].dropna()
    ret = px.pct_change().fillna(0)
    
    # 基准：买入持有
    equity_bh = equity(ret)
    
    print(f"数据下载成功，共{len(px)}个交易日")
    print(f"数据区间：{px.index[0].strftime('%Y-%m-%d')} 至 {px.index[-1].strftime('%Y-%m-%d')}")
    print(f"AAPL价格：{px.iloc[0]:.2f} -> {px.iloc[-1]:.2f}")
    
except Exception as e:
    print(f"数据下载失败：{e}")
    print("请检查网络连接后重试")


## 2. 策略一：SMA动量策略

### 目的
实现简单移动平均线交叉策略，当快线上穿慢线时做多，下穿时平仓。

### 金融含义
SMA交叉是最经典的技术分析信号，基于短期趋势突破长期趋势的逻辑。快慢均线的选择直接影响策略的敏感性和换手率。


In [ ]:
# 策略参数设置
fast = 10
slow = 50
commission_bps = 5
slippage_bps = 2
half_spread_bps = 5

# SMA动量信号
sma_fast = px.rolling(fast).mean()
sma_slow = px.rolling(slow).mean()
signal_sma = (sma_fast > sma_slow).astype(int)

# 严格时序对齐：信号用于下一期交易
pos_sma = shift_exec(signal_sma)

# 计算交易成本
turnover_sma = turnover(pos_sma)
cost_sma = apply_cost(turnover_sma, commission_bps, slippage_bps, half_spread_bps)

# 含成本的策略收益
ret_sma = pos_sma * ret - cost_sma
equity_sma = equity(ret_sma)

print(f"SMA策略构建完成")
print(f"参数：快线{fast}日，慢线{slow}日")
print(f"交易成本：佣金{commission_bps}bps，滑点{slippage_bps}bps，价差{half_spread_bps}bps")
print(f"年化换手率：{turnover_sma.sum() * 252 / len(turnover_sma):.2f}次")


## 3. 策略二：时间序列动量（TSMOM）

### 目的
实现时间序列动量策略，基于价格相对于历史价格的表现决定仓位方向。

### 金融含义
TSMOM通过比较当前价格与过去价格，捕捉中长期趋势。相比SMA交叉，TSMOM更直接地利用价格动量，但对回望期敏感。


In [ ]:
# TSMOM策略参数
lookback = 63  # 约3个月

# TSMOM信号：当前价格 vs 回望期价格
signal_tsmom = (px / px.shift(lookback) - 1 > 0).astype(int)

# 严格时序对齐
pos_tsmom = shift_exec(signal_tsmom)

# 计算交易成本
turnover_tsmom = turnover(pos_tsmom)
cost_tsmom = apply_cost(turnover_tsmom, commission_bps, slippage_bps, half_spread_bps)

# 含成本的策略收益
ret_tsmom = pos_tsmom * ret - cost_tsmom
equity_tsmom = equity(ret_tsmom)

print(f"TSMOM策略构建完成")
print(f"回望期：{lookback}日")
print(f"年化换手率：{turnover_tsmom.sum() * 252 / len(turnover_tsmom):.2f}次")


## 4. 绩效评估与对比

### 目的
计算并对比买入持有、SMA和TSMOM三种策略的关键绩效指标。

### 金融含义
年化收益率衡量策略盈利能力，波动率反映风险水平，Sharpe比率评估风险调整后收益，最大回撤显示策略在最坏情况下的损失。


In [ ]:
# 计算各策略绩效指标
perf_bh = perf(ret)
perf_sma = perf(ret_sma)
perf_tsmom = perf(ret_tsmom)

# 整理为DataFrame便于对比
results = pd.DataFrame({
    'Buy&Hold': perf_bh,
    'SMA': perf_sma,
    'TSMOM': perf_tsmom
}).round(4)

print("策略绩效对比表")
print("=" * 50)
print(results)
print("\n指标解释：")
print("CAGR：年化复合增长率，衡量策略年化收益")
print("Vol：年化波动率，反映策略风险水平") 
print("Sharpe：夏普比率，风险调整后收益指标")
print("MaxDD：最大回撤，策略历史最大亏损幅度")


## 5. 净值曲线对比

### 目的
可视化三种策略的净值曲线，直观展示各策略的表现差异。

### 金融含义
净值曲线是投资者最关心的指标，体现资金增长轨迹。平滑上升的曲线更受青睐，而剧烈波动或深度回撤会影响投资者信心。


In [ ]:
# 净值曲线对比图
plt.figure(figsize=(12, 6))
plt.plot(equity_bh.index, equity_bh.values, label='Buy&Hold', linewidth=2, alpha=0.8)
plt.plot(equity_sma.index, equity_sma.values, label='SMA动量', linewidth=2, alpha=0.8)
plt.plot(equity_tsmom.index, equity_tsmom.values, label='TSMOM', linewidth=2, alpha=0.8)

plt.title('动量策略净值曲线对比', fontsize=14, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('净值', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"最终净值：")
print(f"Buy&Hold: {equity_bh.iloc[-1]:.3f}")
print(f"SMA动量: {equity_sma.iloc[-1]:.3f}")
print(f"TSMOM: {equity_tsmom.iloc[-1]:.3f}")


## 6. 回撤分析

### 目的
分析SMA策略的回撤特征，理解策略风险控制的重要性。

### 金融含义
回撤反映策略在不利市场环境下的损失幅度，是风险管理的核心指标。深度回撤会严重影响投资者心理，导致策略执行困难。


In [ ]:
# 计算SMA策略回撤序列
running_max = equity_sma.cummax()
drawdown = (equity_sma / running_max - 1) * 100

# 绘制回撤图
plt.figure(figsize=(12, 6))
plt.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red', label='回撤区域')
plt.plot(drawdown.index, drawdown.values, color='red', linewidth=1)

plt.title('SMA动量策略回撤分析', fontsize=14, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('回撤 (%)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"SMA策略回撤统计：")
print(f"最大回撤：{drawdown.min():.2f}%")
print(f"平均回撤：{drawdown[drawdown < 0].mean():.2f}%")
print(f"回撤天数占比：{(drawdown < -1).sum() / len(drawdown) * 100:.1f}%")


## 7. 参数敏感性分析

### 目的
测试SMA策略在不同参数组合下的表现，寻找最优参数并评估策略稳健性。

### 金融含义
参数敏感性分析帮助识别过拟合风险，稳健的策略应在合理参数范围内都有不错表现，而非仅在特定参数下有效。


In [ ]:
# 参数网格搜索
fast_values = [5, 10, 20]
slow_values = [30, 50, 100]

# 存储结果的矩阵
sharpe_matrix = np.zeros((len(fast_values), len(slow_values)))

print("参数敏感性分析进行中...")
for i, fast_param in enumerate(fast_values):
    for j, slow_param in enumerate(slow_values):
        if fast_param < slow_param:  # 确保快线 < 慢线
            # 计算信号
            sma_f = px.rolling(fast_param).mean()
            sma_s = px.rolling(slow_param).mean()
            sig = (sma_f > sma_s).astype(int)
            
            # 严格对齐与成本
            pos = shift_exec(sig)
            cost = apply_cost(turnover(pos), commission_bps, slippage_bps, half_spread_bps)
            ret_strategy = pos * ret - cost
            
            # 计算Sharpe
            sharpe_matrix[i, j] = perf(ret_strategy)['Sharpe']
        else:
            sharpe_matrix[i, j] = np.nan

# 转换为DataFrame便于查看
sharpe_df = pd.DataFrame(sharpe_matrix, 
                        index=[f'Fast{f}' for f in fast_values],
                        columns=[f'Slow{s}' for s in slow_values])

print("\nSMA策略参数敏感性分析（Sharpe比率）")
print("=" * 45)
print(sharpe_df.round(3))

# 找出最优参数
best_idx = np.nanargmax(sharpe_matrix)
best_i, best_j = np.unravel_index(best_idx, sharpe_matrix.shape)
print(f"\n最优参数组合：Fast={fast_values[best_i]}, Slow={slow_values[best_j]}")
print(f"最优Sharpe：{sharpe_matrix[best_i, best_j]:.3f}")


## 8. 目标波动率策略（可选）

### 目的
演示目标波动率调整，通过动态杠杆控制策略风险暴露。

### 金融含义
目标波动率策略通过调整仓位大小来稳定策略波动性，在低波动时加杠杆，高波动时减仓，实现更稳定的风险暴露。


In [ ]:
# 目标波动率参数
target_vol = 0.15  # 目标年化波动率15%
vol_window = 20    # 波动率估算窗口
max_leverage = 3   # 最大杠杆倍数

# 计算滚动波动率（年化）
rolling_vol = ret.rolling(vol_window).std() * np.sqrt(252)

# 计算动态杠杆倍数
leverage = (target_vol / rolling_vol).fillna(1)
leverage = np.clip(leverage, 0, max_leverage)  # 限制杠杆范围

# 应用目标波动率到SMA策略
pos_sma_vol = pos_sma * leverage
ret_sma_vol = pos_sma_vol * ret - apply_cost(turnover(pos_sma_vol), commission_bps, slippage_bps, half_spread_bps)
equity_sma_vol = equity(ret_sma_vol)

# 对比图
plt.figure(figsize=(12, 6))
plt.plot(equity_sma.index, equity_sma.values, label='SMA原始', linewidth=2, alpha=0.8)
plt.plot(equity_sma_vol.index, equity_sma_vol.values, label='SMA目标波动率', linewidth=2, alpha=0.8)

plt.title('目标波动率策略对比', fontsize=14, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('净值', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 绩效对比
perf_sma_vol = perf(ret_sma_vol)
vol_comparison = pd.DataFrame({
    'SMA原始': perf_sma,
    'SMA目标波动率': perf_sma_vol
}).round(4)

print("目标波动率策略对比")
print("=" * 30)
print(vol_comparison)
print(f"\n平均杠杆倍数：{leverage.mean():.2f}")
print(f"最大杠杆倍数：{leverage.max():.2f}")


## 9. 数据导出

### 目的
将策略信号、收益和净值数据保存到parquet文件，便于后续分析和复现。

### 金融含义
规范的数据管理是量化研究的基础，保存中间结果有助于策略迭代、回测验证和风险监控。


In [ ]:
# 确保数据目录存在
ensure_dir("data")

# 整理要导出的数据
export_data = pd.DataFrame({
    'price': px,
    'return': ret,
    'signal_sma': signal_sma,
    'signal_tsmom': signal_tsmom,
    'pos_sma': pos_sma,
    'pos_tsmom': pos_tsmom,
    'ret_sma': ret_sma,
    'ret_tsmom': ret_tsmom,
    'equity_bh': equity_bh,
    'equity_sma': equity_sma,
    'equity_tsmom': equity_tsmom,
    'turnover_sma': turnover_sma,
    'turnover_tsmom': turnover_tsmom,
    'cost_sma': cost_sma,
    'cost_tsmom': cost_tsmom
})

# 导出到parquet文件
export_data.to_parquet("data/day4_momentum.parquet")

print("数据导出完成!")
print(f"文件保存路径: data/day4_momentum.parquet")
print(f"数据维度: {export_data.shape}")
print(f"包含字段: {list(export_data.columns)}")


## 总结与要点

### 策略设计要点
1. **时序对齐**：交易信号必须在下一期执行，避免前视偏差
2. **成本控制**：佣金、滑点、价差等交易成本对策略表现影响显著
3. **参数稳健性**：优秀策略应在合理参数范围内都有不错表现

### 动量策略特点
- **SMA交叉**：基于技术分析，参数敏感性较高，换手适中
- **TSMOM**：直接利用价格动量，实现简单，但对回望期敏感
- **目标波动率**：通过动态杠杆控制风险暴露，提升风险调整收益

### 风险管理
- 最大回撤是衡量策略风险的重要指标
- 高换手率会显著增加交易成本，侵蚀策略收益
- 参数过度优化可能导致过拟合，实际交易效果不佳

动量策略是量化交易的重要组成部分，但需要结合严格的风险管理和成本控制才能在实际交易中取得成功。
